### 배포 - npm build 후에 배포

In [1]:
import paramiko

import os
def get_file_list(path):
    return [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

hostname = "maxy.iptime.org"
# hostname = "192.168.10.177"
username = "songs"
password = "thdghqkr74"
port = 8022
remote_path = "/home/songs/Projects/maxy/compose/maxy2-svc"
# SSH 클라이언트 객체 생성
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [2]:

def check_remote_dir(sftp, remote_path):
    """원격 경로가 없으면 계층적으로 생성"""
    dirs = remote_path.strip('/').split('/')
    path = ''
    for d in dirs:
        path += '/' + d
        try:
            sftp.stat(path)  # 존재 여부 확인
        except FileNotFoundError:
            sftp.mkdir(path)
            

def FileUpload(sftp,SourceFile,TargetFile):
    try:
        with sftp.open(TargetFile, "wb") as remote_file, open(SourceFile, "rb") as local_file:
            chunk_size = 1024 * 1024  # 32KB씩 다운로드
            downloaded_size = 0
            while True:
                data = local_file.read(chunk_size)
                if not data:
                    break
                remote_file.write(data)
                downloaded_size += len(data)
                print(f"업로드중: {SourceFile} -> {downloaded_size}")
        print(f"완료: {TargetFile}")
    except Exception as e:
        print(f"FileUpload 오류 발생: {SourceFile}-{TargetFile}-{e}")
        
        
def FolderUpdate(sftp,SourceFolder,TargetFolder):
    try:
        # flist = get_file_list(f"{SourceFolder}")
        flist = os.listdir(f"{SourceFolder}")
        for file in flist:
            if file == "." or file == "..":
                continue
            if os.path.isdir(f"{SourceFolder}/{file}"):
                check_remote_dir(sftp,f"{TargetFolder}/{file}")
                FolderUpdate(sftp,f"{SourceFolder}/{file}",f"{TargetFolder}/{file}")
                continue
            SourceFile = f"{SourceFolder}/{file}"
            TargetFile = f"{TargetFolder}/{file}"
            FileUpload(sftp,SourceFile,TargetFile)
    except Exception as e:
        print(f"FolderUpdate 오류 발생: {SourceFolder}-{TargetFolder}-{e}")

In [3]:
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
try:
    ssh_client.connect(hostname, port, username, password)
    TargetPath = f"{remote_path}"
    SourcePath = f"."
    # SFTP 클라이언트 생성
    sftp = ssh_client.open_sftp()
    FileUploadDir = ["common"]
    for folder in FileUploadDir:
        check_remote_dir(sftp,f"{TargetPath}/{folder}")
        FolderUpdate(sftp,f"{SourcePath}/{folder}",f"{TargetPath}/{folder}")    
    UploadFiles = [
        (f"{SourcePath}/maxy_core.py",f"{TargetPath}/maxy_core.py")
    ]
    for SourceFile,TargetFile in UploadFiles:
        FileUpload(sftp,SourceFile,TargetFile)
    sftp.close()
except Exception as e:
    print(f"오류 발생: {e}")
finally:
    ssh_client.close()

업로드중: ./common/maxy_query.py -> 66483
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/maxy_query.py
업로드중: ./common/maxy_common.py -> 6073
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/maxy_common.py
업로드중: ./common/__pycache__/maxy_crypto.cpython-312.pyc -> 5760
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/__pycache__/maxy_crypto.cpython-312.pyc
업로드중: ./common/__pycache__/maxy_common.cpython-312.pyc -> 7306
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/__pycache__/maxy_common.cpython-312.pyc
업로드중: ./common/maxy_crypto.py -> 2427
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/maxy_crypto.py
업로드중: ./common/hmariadb.py -> 1457
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/hmariadb.py
업로드중: ./common/maxy_opensearch.py -> 60911
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/maxy_opensearch.py
업로드중: ./common/maxy_clickhouse.py -> 2111
완료: /home/songs/Projects/maxy/compose/maxy2-svc/common/maxy_clickhouse.py
업로드중: ./maxy_core.py -> 74

In [4]:
# flist = get_file_list(f".next")
# os.listdir(path=".next")